In [28]:
#Importacion de las librerias 
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from surprise import KNNWithMeans, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split

In [29]:
# Creación de una constante que tomara la base de datos
PATH = 'data.csv'

# Import Data

In [30]:
# Lectura de la base de datos desde la constante anteriormente creada 
df = pd.read_csv(PATH)
df.shape

(100000, 10)

In [31]:
# Visualizamos los primeros datos de nuestra base de datos
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,655,52,4,11482,300,4,8,2012,94,7
1,2713,90,3,6479,469,1,8,2012,33,5
2,409,17,2,25472,435,1,12,2001,196,4
3,1150,234,10,23950,529,2,23,2019,79,2
4,2424,390,5,13046,395,2,20,2010,200,4


# Method 1

In [32]:
# generación de una tabla dinámica o pivote con el id_bock columna, reader_id 
# como index y book_rating como valor

pt_df = df.pivot_table(
    columns = 'book_id',
    index = 'reader_id',
    values = 'book_rating'
).fillna(0)

# Convercion matriz crs
mat = pt_df.values
mat = csr_matrix(mat)

In [33]:
# Normalización de la salida de pred_ratings, pasando como parámetro la lista de puntuación

def normalize(pred_ratings):
    return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())

In [34]:
# Función cálculo de descomposición de un solo valor de la matriz de entrada dados n_factores. 
# generará y normalizará las predicciones de valoración de los usuarios.

def generate_prediction_df(mat, pt_df, n_factors):
    if not 1 <= n_factors < min(mat.shape):
        raise ValueError("Must be 1 <= n_factors < min(mat.shape)")
        
    # Matriz de factorizacion
    u, s, v = svds(mat, k = n_factors)
    s = np.diag(s)

    # Calculo de prediccion de puntuacion
    pred_ratings = np.dot(np.dot(u, s), v) 
    pred_ratings = normalize(pred_ratings)
    
    # Convercion a df
    pred_df = pd.DataFrame(
        pred_ratings,
        columns = pt_df.columns,
        index = list(pt_df.index)
    ).transpose()
    return pred_df

In [35]:
%time pred_df = generate_prediction_df(mat, pt_df, 10)

Wall time: 1.51 s


In [36]:
# La función realza los cálculos de los ítems recomendado a los usuarios, mediante la tabla de predicción , el usr_id  
# y el número de ariticulos el cual se va a recomendar


def recommend_items(pred_df, usr_id, n_recs):

    usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
    rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
    return rec_df

In [37]:
# Se utiliza la función mandando por parámetros la tabla de predicción, el id del usuario al cual se va a realizar la 
# recomendación y el número de ítems o artículos que se le van a recomendar

recommend_items(pred_df, 5, 5)

,book_id,sim
0,1108,0.143244
1,1839,0.140945
2,725,0.140516
3,1897,0.140497
4,1341,0.140086


## Method 2

In [38]:
# generación de una tabla dinámica o pivote con el id_bock columna, author_id como index y 
# book_rating como valor

pt_df = df.pivot_table(
    columns = 'book_id', 
    index = 'author_id', 
    values = 'book_rating'
).fillna(0)

In [39]:
# Función de buscar lectores similares, para esta función se ingresará la tabla de pivote, 
# el id del lector y el número de recomendaciones

def find_similar_readers(pt_df, reader_id, n_recs):
  
    # Primero es necesario separa a los lectores de interés de los otros lectores para esto 
    # realizaremos comparaciones
    reader = pt_df[pt_df.index == reader_id]
    other_readers = pt_df[pt_df.index != reader_id]

    # Obtenemos la similitud de nuestro lector en comparación a los demás lectores
    sim = cosine_similarity(reader, other_readers)[0].tolist()
    idx = other_readers.index.tolist()

    
    # Creación de un diccionario de similitud en donde se guardarán todas las recomendaciones 
    # para nuestro lector de interés
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)

    similar_readers = idx_sim[:n_recs]
    readers = [rdr[0] for rdr in similar_readers]

    return readers

In [40]:
# Para el usuario de la función es necesario ingresar la tabla pivotal, el id del lector y 
#el número de recomendaciones total que se le dará al lector

find_similar_readers(pt_df = pt_df, reader_id = 226, n_recs = 5)

[319, 191, 145, 162, 212]

# Method 3

In [41]:
# Tomamos las columnas: id del lector ,id del libro y puntuación del libro
rdf = df[['reader_id', 'book_id', 'book_rating']]

In [42]:
# leemos los datos del dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rdf, reader)

In [43]:
# Realizamos la partición de los datos para el entrenamiento y prueba del modelo
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [44]:
# Hacemos la filtración colaborativa de los usuarios o el artículo en este caso el libro
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [45]:
# Ejecutamos el entrenamiento con los datos de prueba
test_pred = algo.test(testset)

In [46]:
# Calculamos el RMSE de nuestros datos
accuracy.rmse(test_pred, verbose=True)

RMSE: 2.9306


2.9306185721359865

In [47]:
# Visualizamos los datos de la prediccion
algo.predict(uid = 10, iid = 43)

Prediction(uid=10, iid=43, r_ui=None, est=5, details={'actual_k': 0, 'was_impossible': False})